In [ ]:
onnx_model_path = "./model/model.onnx"

In [ ]:
import onnxruntime as ort
from transformers import BertTokenizer
import numpy as np

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load the ONNX model
#qnn session
options = ort.SessionOptions()
# options.add_session_config_entry("session.disable_cpu_ep_fallback", "1")
session = ort.InferenceSession(onnx_model_path,
    sess_options=options,
    providers=["QNNExecutionProvider"],
    provider_options=[{"backend_path": "QnnHtp.dll"}])

def areParaphrases(sentence1, sentence2):
    # Tokenize the input sentences
    inputs = tokenizer(sentence1, sentence2, return_tensors='np', padding='max_length', truncation=True, max_length=128)

    # Convert inputs to numpy arrays
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    token_type_ids = inputs['token_type_ids']

    # Run inference
    outputs = session.run(None, {
        'input_ids': input_ids.astype(np.int64),
        'attention_mask': attention_mask.astype(np.int64),
        'token_type_ids': token_type_ids.astype(np.int64)
    })

    # Get the prediction
    logits = outputs[0]
    predicted_label = np.argmax(logits, axis=1)
    return predicted_label == 1

In [ ]:
# Prepare the input data
sentence1 = "The company Hugging Face is based in New York City."
sentence2 = "Hugging Face's headquarters are situated in NYC."
sentence3 = "New York City is big."

# Interpret the result
if areParaphrases(sentence1, sentence2):
    print("Expected: The sentences are paraphrases.")
else:
    print("Not expected: The sentences are not paraphrases.")

if areParaphrases(sentence1, sentence3):
    print("Not expected: The sentences are paraphrases.")
else:
    print("Expected: The sentences are not paraphrases.")